# The Autonomous Vehicle Co-pilot
Here is a chatbot, empowered by GPT-3.5, running to help the vehicle driver. 

He is set as a co-pilot, analyzing the environment based on other multi-modality input, which has been processed as text so he can reason and identify potential risk. 


## Setup
Notice: if you tried to run this program on your linux server, 
and you are in China,
make sure that you opened the vpn. 
Otherwise the api account may be forbiddened!

In [ ]:
import os
import openai
os.environ["http_proxy"]="127.0.0.1:10080"
os.environ["https_proxy"]="127.0.0.1:10080"

def read_from_file(filepath):
    with open(filepath, 'r') as file:
        return file.read().strip()

openai.api_key  = read_from_file('key/apikey.cn')

TEMPERATURE = 0

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo-16k-0613"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= TEMPERATURE, # this is the degree of randomness of the model's output
    )
    return response
    # return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo",temp = TEMPERATURE):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature= temp, # this is the degree of randomness of the model's output
    )
    # print(str(response.choices[0].message["content"]))
    return response.choices[0].message["content"]

## test on the api

In [ ]:
blind_guide = """ 
We are currently driving on a city street filled with green plants, with bicycle lanes on both sides of the road. On the left bicycle lane, there are three cyclists moving slowly, while on the right side, there are two cyclists riding at a faster pace.
Next, there is a spacious pedestrian walkway, with five pedestrians walking on the left side and four pedestrians on the right side, two of whom are jogging. Each side of the street is lined with banyan trees, providing ample shade for pedestrians to cool off.
We are heading towards an intersection. The traffic signal at the intersection is displaying a green light, indicating that we can proceed. Each direction at the intersection has only one lane, clearly marked. In front of us, there is a blue sedan driving at a moderate speed."""
chat_history = [
    { 'role':'system','content': blind_guide},
    {'role':'system', 'content':'You are a co-pilot AI assistant designed to help drivers understand their surroundings.'},
    {'role':'user', 'content':'What is our current situation on the road?'},
]

# test_reponse = get_completion_from_messages(chat_history)

# Data Integration
 Firstly, you'll need a way to convert the input from the vehicle's sensors (like LIDAR, RADAR, cameras, etc.) into a format that the chatbot can understand. This might involve using object detection algorithms to identify objects and their positions, then converting this data into a text description of the scene.


## buggy chatbot here
I need to find a good programming way of coding here

In [ ]:
# def chat_with_assistant(user_input, context):
#     # Append the user input to the context
#     context.append({'role':'user', 'content':f"{user_input}"})
    
#     # Get the assistant's response
#     response = get_completion_from_messages(context)
    
#     # Append the assistant's response to the context
#     context.append({'role':'assistant', 'content':f"{response}"})
    
#     return context


# # Chat with the assistant
# context = chat_with_assistant('Hello, assistant!', chat_history)
# print(context)

# # Continue the conversation
# context = chat_with_assistant('Tell me a joke', context)
# print(context)


ChatGPT, based on the incoming road scenario data, I need you to assess the situation and provide us with critical information. 
## input
The road scenario data will be provided in {'context'}, it contains the continuous frame information of video from dashcam in the vehicle. The autonomous system provides you with information on road layout, people's locations, their distances and angles from the dashcam, the surfaces they are on, and the confidence level of each detection.
## output 
Your analysis should provide:
- an understanding of the context complexity (low, medium, or high), 
- the number of persons, cars, and bikes present in the scene, 
- an array detailing the danger level for each person, if applicable.
The output should be markdown format

This analysis should be grounded in the current context and make predictions for the short term future to assist in autonomous driving. The key goal here is to identify potential risks to enhance the safety and efficiency of our journey.

"You are a co-pilot AI designed to analyze road scenario data and provide critical information about the situation. Based on the video frames from the dashcam and the data provided by the autonomous system, you will assess the context complexity, count the number of persons, cars, and bikes in the scene, and evaluate the danger level for each person. Your goal is to identify potential risks to enhance the safety and efficiency of the journey. The output should be in markdown format.

In [ ]:
sys_prompt = read_from_file("key/sys_prompt.md")
usr_prompt = read_from_file("key/usr_prompt.md")

def get_sys_msg(content:str): 
    system_msg = {"role": "system", "content": content}
    return  system_msg
def get_usr_msg(content:str): 
    msg = {"role": "user", "content": content}
    return msg


In [ ]:
data = """INFO of 0016:
road 0 is at middle_down
sidewalk 1 is at right_down
sidewalk 2 is at right_down
person 0 is at middle_down
The [distance,angle] from person 0 to our dashcam is: [very close,89.97960036997033]
person 1 is at left_down
The [distance,angle] from person 1 to our dashcam is: [very close,-89.98811258767391]
person 2 is at left_down
The [distance,angle] from person 2 to our dashcam is: [very close,-89.98749824583163]
person 3 is at middle_down
The [distance,angle] from person 3 to our dashcam is: [very close,89.96852785395733]
person 4 is at left_down
The [distance,angle] from person 4 to our dashcam is: [very close,-89.99328360812777]
person 5 is at left_down
The [distance,angle] from person 5 to our dashcam is: [very close,-89.99173842984247]
person 6 is at left_down
The [distance,angle] from person 6 to our dashcam is: [very close,-89.99199265188705]
person 7 is at left_down
The [distance,angle] from person 7 to our dashcam is: [very close,-89.99241327380344]
Person 0 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [1151.6272   683.91156 1259.8259  1029.4976 ]
Person 1 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [520.33466 722.2189  556.2974  830.36035]
Person 2 is on the road 0, sidewalk 1, sidewalk 2,his/her bbox is [544.58014 737.0274  573.4954  827.68024]
Person 3 is on the road 0,his/her bbox is [1110.2614  723.2576 1128.289   741.3161]
Person 4 is on the road 0,his/her bbox is [205.18512 686.6346  222.12915 713.3797 ]
Person 5 is on the road 0, sidewalk 1,his/her bbox is [343.09924 702.2559  363.39508 733.56   ]
Person 6 is on the road 0,his/her bbox is [324.6748  691.9793  343.29242 718.48505]
Person 7 is on the road 0,his/her bbox is [291.81046 690.33636 306.7418  715.5451 ]
number of Surface mask, Road&sidewalk, People 's mask, actural people: (9, 3, 9, 8)"""

In [ ]:
messages = [
    get_sys_msg(sys_prompt),
    get_usr_msg(usr_prompt),
    {"role": "user", "content": f"here's the current road scenario data: {data}"},
]
# response = get_completion_from_messages(messages)

Great, now I have a working chatGPT api as below:{{{
messages = [
    get_sys_msg(sys_prompt),
    get_usr_msg(usr_prompt),
    {"role": "user", "content": f"here's the current road scenario data: {frame_data}"},
]
response = get_completion_from_messages(messages)
}}}
For raw data stored in folder raw_data/ , there are info of video as below:
- raw_data/Info_Video_0001.txt
- raw_data/Info_Video_0002.txt
...
### task: I want you provide a double for-loop for me, here is the pesudo code:

for all the Info txt in raw_data folder{
    for each frame in the txt{
        read the frame info into variable 'frame_data'
        response = get_completion_from_messages
        append the response into 'LLM_data/Result_Video_xxxx', here the xxxx is current video id
    }
}


In [ ]:
def read_frames_from_raw_file(file_path):
    frames = []
    current_frame_data = ''
    reading_frame = False

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('INFO of'):
                if reading_frame:
                    frames.append(current_frame_data.strip())
                    current_frame_data = ''
                reading_frame = True

            if reading_frame:
                current_frame_data += line

    # Adding the last frame if any
    if current_frame_data:
        frames.append(current_frame_data.strip())

    return frames



In [ ]:
import os
import re
from pathlib import Path

raw_data_folder = "raw_data/"

output_folder = "LLM_data/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(raw_data_folder):
    if filename.startswith("Info_Video_") and filename.endswith(".txt"):
        # the id may be 'bd99', '0001', 'eb61'
        video_id = filename[-8:-4]
        output_path = Path(f"LLM_data/Result_Video_{video_id}.txt")
        
        frames = read_frames_from_raw_file(os.path.join(raw_data_folder, filename))
        start_frame = 0
        if output_path.exists():
            print(f"Already handled {output_path}")
            # Read the existing frames that have been processed
            # with open(output_path, 'r') as result_file:
            #     processed_frames = re.findall(r'### Frame (\d{4}) Analysis', result_file.read())
            #     start_frame = len(processed_frames) # Start from the next unprocessed frame
            #     frames = frames[start_frame:] # Excl
            #     if frames is None:
            #         continue
            continue
        with open(os.path.join(raw_data_folder, filename), 'r') as raw_file:
            with open(os.path.join(output_folder, f"Result_Video_{video_id}.txt"), 'a') as result_file:
                print(f"Analyzing video {video_id} starting from frame {start_frame}")

                for frame_data in frames:
                    messages = [
                        get_sys_msg(sys_prompt), 
                        get_usr_msg(usr_prompt), 
                        {"role": "user", "content": f"here's the current road scenario data: {frame_data}"},
                    ]
                    response = get_completion_from_messages(messages)
                    
                    result_file.write(response + '\n')
                    
        print(f"Analysis completed for video {video_id}")

print("All videos processed!")


In [ ]:
def read_frames_from_mot_record(file_path, frames_per_segment=5):
    segments = []
    current_segment_data = ''
    frame_count = 0
    delimiter = "====================================================="

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('At frame'):  # assuming each new frame starts with 'At frame'
                frame_count += 1
                
            if line.strip() == delimiter:
                continue  # skip delimiter lines

            current_segment_data += line

            if frame_count == frames_per_segment:
                segments.append(current_segment_data.strip())
                current_segment_data = ''
                frame_count = 0

    # Adding the last segment if any
    if current_segment_data:
        segments.append(current_segment_data.strip())

    return segments

seg = read_frames_from_mot_record("mot_archive_0828/mot_car_b1c9c847-3bda4659.txt")
seg

In [ ]:
def read_file(file_path):

    with open(file_path, 'r') as file:
        return file


In [26]:
def read_frames_from_mot_record(file_path, frames_per_segment=5):
    """
    Splits the text into segments, each containing data for a specified number of frames.
    """
    # Splitting the text by the starting phrase 'At frame '
    with open(file_path, 'r') as file:
        text = file.read()

    sentences = text.split('At frame ')
    
    segments = []
    current_segment = []
    current_frame = None
    
    for sentence in sentences:  
        # Extract frame number from the sentence
        frame_num = int(sentence.split(',')[0].split(' ')[0])
        
        # If current frame is None or is the same as the last frame, continue appending to the current segment
        if current_frame is None or frame_num == current_frame:
            current_segment.append('At frame ' + sentence)
        else:
            # If the frame number has changed, append the current segment to segments and start a new one
            segments.append('\n'.join(current_segment))
            current_segment = ['At frame ' + sentence]
        
        current_frame = frame_num

        # If the length of the current segment reaches the specified frames_per_segment, append it to segments
        if len(set([int(s.split(' ')[2].split(',')[0]) for s in current_segment])) == frames_per_segment:
            # segments.append('\n'.join(current_segment))
            current_segment = []

    # Append any remaining data to segments
    if current_segment:
        segments.append('\n'.join(current_segment))

    return segments

# Segment the content using the improved function
improved_segments_v2 = read_frames_from_mot_record("mot_archive_0828/mot_car_b1c9c847-3bda4659.txt")
len(improved_segments_v2), improved_segments_v2[0], improved_segments_v2[1]  # Displaying the length and the first two segments for verification

print(improved_segments_v2[4])


At frame 4, car 1 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, car 2 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, car 3 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, car 8 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, car 9 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, car 5 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, person 4 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, truck 7 is located at coordinates [361.46, 292.77] with a bounding box of width 403.93 and height 284.62.
 
At frame 4, person 6 is located at coordinates [361

这个是干什么的，我忘记为什么写他了

In [ ]:
"Based on the following sequence of frames, provide a concise description of the road scene, capturing significant events or movements of people and vehicles:"


c

In [ ]:
def extract_information(input_text):
    # Find the position of "Therefore" in the text
    index = input_text.find("Therefore")
    
    # If "Therefore" is found, extract the text after it
    if index != -1:
        information_after_therefore = input_text[index:]
    else:
        information_after_therefore = "Keyword 'Therefore' not found in the text."

    return information_after_therefore

def write_to_file(output_text, file_name='output.txt'):
    with open(file_name, 'w') as file:
        file.write(output_text)

if __name__ == "__main__":
    # Read input text from a file
    with open('input.txt', 'r') as file:
        input_text = file.read()

    # Extract the information after "Therefore"
    output_text = extract_information(input_text)

    # Write the output to another text file
    write_to_file(output_text)

    print(f"Information after 'Therefore' has been written to {output_file_name}")


2. **Contextual Understanding:** Your chatbot should be capable of maintaining and updating a model of the car's surroundings based on this information. For example, it should keep track of the positions of other vehicles, pedestrians, and road features, updating these as new data comes in.

3. **Real-Time Interaction:** The chatbot should be capable of generating responses in real-time. Delays could be dangerous in a driving context.

4. **Proactive Alerting:** Your co-pilot chatbot should not just react to the driver's queries but should also proactively provide alerts and advice. For example, if it detects a pedestrian stepping onto the road, it should immediately alert the driver.

5. **Simulated Training:** Consider using simulated environments to train and test your chatbot before deploying it in a real vehicle. This can help you ensure that the chatbot behaves correctly in a wide range of situations.

6. **Safety Precautions:** Since the bot is serving in a co-pilot role, it should prioritize safety and assume the driver may be unaware of potential dangers. It should also be prepared to handle ambiguous situations by advising caution.

7. **User Customization:** Different drivers may have different preferences for how much information they want to receive, what kind of language the co-pilot uses, etc. Consider allowing users to customize the chatbot's behavior to some extent.

8. **Integration with Vehicle's Systems:** If possible, the chatbot could also be integrated with the vehicle's systems to provide even more functionality. For example, it could adjust the vehicle's speed or even take control in an emergency. However, this would require a high level of reliability and many legal and ethical considerations.